In [1]:
#importing all the required libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the complete patient file
df_patient = pd.read_csv('DM2_DR_EVENTS_dedup.tsv')

#Importing the  file for the FENOFIBRATE prescription
df_feno = pd.read_csv('FENO_DR_EVENTS_dedup.tsv')

#Importing the  file for the MONTELUKAST prescription
df_mont = pd.read_csv('MONT_DR_EVENTS_dedup.tsv')

##Importing the  file for the ZOLPIDEM prescription
df_zolp = pd.read_csv('ZOLP_DR_EVENTS_dedup.tsv')

#Renaming the columns in the medicine files for consistency as in the problem statement we have rxlen but in some 
#of the files we have days.
#Keeping consistent column names

df_feno.rename(columns = {'days' : 'rxlen'}, inplace = True)
df_zolp.rename(columns = {'days' : 'rxlen'}, inplace = True)

# In order to identify what is the type of medicine that is being given, added a drug column that stores the medicine name
#for further analysis.

df_feno['drug'] = 'feno'
df_mont['drug'] = 'mont'
df_zolp['drug'] = 'zolp'


In [3]:
#Concatinating the 3 medicine dataframes into 1 for further detailed analysis
frames = [df_feno, df_mont, df_zolp]
df_drugs=pd.concat(frames)

# If a patient is already taking medicines, more than 30 days before being diagnosed with DM2, maybe it is related with other
# symptoms and they are not vital to our analysis of studying the effect of drugs on protective development of DR after DM2

df_drugs = df_drugs[df_drugs.time > -30]
df_drugs = df_drugs.reset_index().drop(columns = 'index')

#Creating dummy variable for the drug column to distinguish between the different treatment groups and study the 
#roles of these medicines for their protective effect on development of DR. 

df_drugs = pd.get_dummies(df_drugs, columns = ['drug'])
df_drugs = df_drugs.reset_index().drop(columns = ['index'])

# Sorting the combined medicine file by ptid and time and resetting the index too.
df_drugs.sort_values(by=['ptid', 'time'], inplace = True)
df_drugs = df_drugs.reset_index().drop(columns = ['index'])

#Checking for the top 10 rows 
df_drugs.head()

,ptid,time,rxlen,drug_feno,drug_mont,drug_zolp
0,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,192,30.00000,0,0,1
1,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,215,30.00000,0,0,1
2,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,298,30.00000,0,0,1
3,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,-7,30.00000,0,1,0
4,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,20,90.00000,0,1,0


In [4]:
# Renaming time column of df_drugs to start
df_drugs.rename(columns = {'time': 'start'}, inplace = True)

# Merging the medicine file and patient file using a left outer join on ptid columns of both df_drugs and df_patient to get the
#age, event and gender column for each of the patient on a prescription

df_merged = df_drugs.merge(df_patient, how= 'left', on = 'ptid')
df_merged = pd.get_dummies(df_merged, columns = ['gender'])
df_merged.head()

,ptid,start,rxlen,drug_feno,drug_mont,drug_zolp,time,event,age,gender_F,gender_M,gender_O
0,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,192,30.00000,0,0,1,1057,1,65,0,1,0
1,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,215,30.00000,0,0,1,1057,1,65,0,1,0
2,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,298,30.00000,0,0,1,1057,1,65,0,1,0
3,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,-7,30.00000,0,1,0,697,0,65,1,0,0
4,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,20,90.00000,0,1,0,697,0,65,1,0,0


In [5]:
def create_survival_data(ptid_list):
    df_final = pd.DataFrame()
    for ptid in ptid_list:
        df_example=df_merged[df_merged.ptid== ptid]
        df_example.reset_index(drop=True,inplace=True)
        
    
        df=pd.DataFrame()
        count = 0
        for i in range(0,len(df_example)):
            df=df.append(df_example.loc[i],ignore_index=True)
            if i==len(df_example)-1:
                break
            if (df_example.loc[i+1,'start']-df_example.loc[i,'start'])>df_example.loc[i,'rxlen']:
                df=df.append(df_example.loc[i],ignore_index=True)
                count+=1
                df.loc[count+i,'start']=df.loc[count+i,'start']+df.loc[count+i,'rxlen']
                df.loc[count+i,'drug_mont']=0
                df.loc[count+i,'drug_feno']=0
                df.loc[count+i,'drug_zolp']=0
                df.loc[count+i,'rxlen'] = 0
        df['stop']=df['start'].shift(-1)
        df['stop'].iloc[-1]=df['time'].iloc[0]
        df.drop(columns = 'time', inplace = True)
        df = df[['ptid', 'start', 'stop', 'rxlen', 'drug_feno', 'drug_mont', 'drug_zolp', 'age',
                 'gender_F','gender_M','gender_O', 'event']]

        if df['start'].iloc[0] >0:
            df.loc[-1] = [ptid, 0, df.loc[0,'start'], np.nan, 0,0,0, df.loc[1,'age'],df.loc[1,'gender_F'],
                          df.loc[1,'gender_M'],df.loc[1,'gender_O'], df.loc[1,'event']]

            df.index = df.index + 1  # shifting index
            df.sort_index(inplace=True) 

        if df['event'].iloc[-1] == 1:
            df.event[:-1] = 0 
        
        df_final = pd.concat([df_final,df], ignore_index = True)
    return df_final

In [6]:
ptid_list = df_merged.ptid.unique()[:10].tolist()

In [7]:
df_final = create_survival_data(ptid_list)

In [8]:
df_final.head(21)

,ptid,start,stop,rxlen,drug_feno,drug_mont,drug_zolp,age,gender_F,gender_M,gender_O,event
0,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,0.00000,192.00000,nan,0.00000,0.00000,0.00000,65,0.00000,1.00000,0.00000,0.00000
1,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,192.00000,215.00000,30.00000,0.00000,0.00000,1.00000,65,0.00000,1.00000,0.00000,0.00000
2,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,215.00000,245.00000,30.00000,0.00000,0.00000,1.00000,65,0.00000,1.00000,0.00000,0.00000
3,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,245.00000,298.00000,0.00000,0.00000,0.00000,0.00000,65,0.00000,1.00000,0.00000,0.00000
4,000019eb05045d9c5a1843eebd4cf9f68d550f466091e88dffaab391ff63f54b,298.00000,1057.00000,30.00000,0.00000,0.00000,1.00000,65,0.00000,1.00000,0.00000,1.00000
5,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,-7.00000,20.00000,30.00000,0.00000,1.00000,0.00000,65,1.00000,0.00000,0.00000,0.00000
6,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,20.00000,110.00000,90.00000,0.00000,1.00000,0.00000,65,1.00000,0.00000,0.00000,0.00000
7,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,110.00000,113.00000,0.00000,0.00000,0.00000,0.00000,65,1.00000,0.00000,0.00000,0.00000
8,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,113.00000,203.00000,90.00000,0.00000,1.00000,0.00000,65,1.00000,0.00000,0.00000,0.00000
9,0001480f995ecb1fb19c2f6cce0d3b1730f6538213763c0954d3fc3fda6f19d9,203.00000,209.00000,0.00000,0.00000,0.00000,0.00000,65,1.00000,0.00000,0.00000,0.00000
